# Policy Knowledge Base

This notebook creates the comprehensive Cigna policy knowledge base using Cortex Complete and sets up Cortex Search for policy integration.

**Prerequisites:**
- **MUST run all previous notebooks first**:
  - `01_Data_Views_for_Agent_Workflows.ipynb` 
  - `02_Builder_Agent_Development.ipynb`
  - `03_Builder_Agent_Configuration.ipynb`
  - `04_Insurance_Agent_Development.ipynb`
- Both Builder and Insurance Agents working
- Cortex Complete and Cortex Search enabled

**What this builds:**
- Generate comprehensive Cigna policy documents using Cortex Complete
- Set up Cortex Search service over policy documents
- Create policy search functions for agent integration
- Enhance Insurance Agent with policy document references
- Test policy-backed agent responses

**OUTCOME:** Policy knowledge base ready for dual-agent orchestration workflow (Phase 6).


## Step 1: Generate Cigna Policy Documents

Create comprehensive, realistic Cigna policy documents using Cortex Complete.


In [ ]:
-- Create policy documents table
CREATE OR REPLACE TABLE CLAIMS_DEMO.PUBLIC.CIGNA_POLICY_DOCUMENTS (
    DOCUMENT_ID VARCHAR(50) PRIMARY KEY,
    DOCUMENT_TITLE VARCHAR(200),
    DOCUMENT_TYPE VARCHAR(100),
    DOCUMENT_CONTENT TEXT,
    SECTION_NUMBER VARCHAR(20),
    POLICY_AREA VARCHAR(100),
    CREATED_DATE TIMESTAMP DEFAULT CURRENT_TIMESTAMP()
);


In [ ]:
-- Generate and store laboratory services policy
INSERT INTO CLAIMS_DEMO.PUBLIC.CIGNA_POLICY_DOCUMENTS (DOCUMENT_ID, DOCUMENT_TITLE, DOCUMENT_TYPE, DOCUMENT_CONTENT, SECTION_NUMBER, POLICY_AREA) VALUES (
    'POLICY_LAB_V1',
    'Cigna Laboratory Services Coverage Policy',
    'SPECIALTY_POLICY',
    'SECTION 4.1 - LABORATORY SERVICES

Coverage Criteria: Laboratory services covered when medically necessary and ordered by healthcare provider. Must be performed at Cigna-approved facility.

Covered Procedures:
- 85025 (Complete Blood Count): Covered 100% for preventive care, copay may apply for diagnostic
- 80053 (Comprehensive Metabolic Panel): Annual preventive coverage, quarterly for diabetes monitoring  
- 36415 (Blood Draw/Venipuncture): Standard coverage with lab services

Frequency Limits:
- CBC: 2 per year for preventive care, additional as medically necessary
- CMP: Annual for preventive, quarterly for diabetes patients
- Genetic testing: Requires prior authorization

Prior Authorization: Not required for routine lab work. Required for experimental genetic testing over $500.

Medical Necessity: Must be ordered by physician for specific medical indication. Routine screening allowed per preventive care guidelines.

Documentation Requirements: Physician order with appropriate diagnosis code. ICD-10 codes must justify procedure medical necessity.

Exclusions: Experimental genetic testing, research laboratory work, tests not ordered by licensed provider.',
    'SECTION_4.1',
    'LABORATORY_SERVICES'
);


In [ ]:
-- Generate and store emergency services policy
INSERT INTO CLAIMS_DEMO.PUBLIC.CIGNA_POLICY_DOCUMENTS (DOCUMENT_ID, DOCUMENT_TITLE, DOCUMENT_TYPE, DOCUMENT_CONTENT, SECTION_NUMBER, POLICY_AREA) VALUES (
    'POLICY_EMERGENCY_V1',
    'Cigna Emergency Services Coverage Policy',
    'SPECIALTY_POLICY',
    'SECTION 4.2 - EMERGENCY SERVICES

Coverage Criteria: Emergency services covered 24/7 without prior authorization for true medical emergencies meeting prudent layperson standard.

Covered Procedures:
- 99281 (ED Visit - Problem Focused): Low complexity emergency visits
- 99282 (ED Visit - Expanded): Moderate complexity emergency visits  
- 99283 (ED Visit - Detailed): Moderate complexity with comprehensive exam
- 99284 (ED Visit - Comprehensive): High complexity emergency visits
- 99285 (ED Visit - Comprehensive High): Critical/complex emergency care

Prior Authorization: NOT required for true emergency situations. Cigna cannot require prior auth for emergency care.

Prudent Layperson Standard: Coverage based on whether average person with reasonable knowledge would believe emergency care was needed.

Network Requirements: Emergency care covered at same level regardless of network status. Out-of-network emergency providers covered at in-network rates.

Non-Emergency Use: If ED used for non-emergency care, member may be responsible for additional cost-sharing.

Documentation Requirements: Emergency department triage notes, physician assessment, discharge instructions. Must document emergency nature of visit.',
    'SECTION_4.2',
    'EMERGENCY_SERVICES'
);


In [ ]:
-- Generate and store cardiology services policy
INSERT INTO CLAIMS_DEMO.PUBLIC.CIGNA_POLICY_DOCUMENTS (DOCUMENT_ID, DOCUMENT_TITLE, DOCUMENT_TYPE, DOCUMENT_CONTENT, SECTION_NUMBER, POLICY_AREA) VALUES (
    'POLICY_CARDIOLOGY_V1',
    'Cigna Cardiology Services Coverage Policy',
    'SPECIALTY_POLICY',
    'SECTION 6.1 - CARDIOLOGY SERVICES

Coverage Criteria: Cardiology services covered when medically necessary for diagnostic, monitoring, or treatment purposes. Must be provided by qualified cardiologist or under cardiologist supervision.

Covered Procedures:
- 93005 (Electrocardiogram): Covered for diagnostic purposes when cardiac symptoms present
- 93010 (EKG Interpretation): Professional interpretation of EKG results
- 93015 (Cardiovascular Stress Test): Annual screening or as medically indicated

Prior Authorization: REQUIRED for all cardiology services except emergency EKG. Must submit clinical indication and symptom documentation.

Medical Necessity Requirements:
- Must have cardiac symptoms (chest pain, palpitations, shortness of breath) OR
- Risk factors requiring cardiac screening (family history, diabetes, hypertension) OR
- Follow-up for known cardiac condition

Frequency Limits:
- EKG: As medically necessary, no specific limit for symptomatic patients
- Stress Tests: Annual for high-risk patients, as clinically indicated otherwise
- Screening in asymptomatic low-risk patients under age 40: Generally NOT covered

Documentation Requirements: Cardiology consultation notes, clinical indication documentation, symptom timeline, risk factor assessment.

Exclusions: Screening in asymptomatic low-risk patients under 40, research studies, experimental cardiac procedures.',
    'SECTION_6.1',
    'CARDIOLOGY_SERVICES'  
);


## Step 2: Set Up Cortex Search Service

Create Cortex Search service over policy documents for semantic search capabilities.


In [ ]:
-- Create Cortex Search service over policy documents
CREATE OR REPLACE CORTEX SEARCH SERVICE CIGNA_POLICY_SEARCH
ON DOCUMENT_CONTENT
ATTRIBUTES DOCUMENT_TITLE, POLICY_AREA, SECTION_NUMBER
WAREHOUSE = DEMO_WH
TARGET_LAG = '1 minute'
AS (
    SELECT 
        DOCUMENT_CONTENT,
        DOCUMENT_TITLE,
        POLICY_AREA,
        SECTION_NUMBER,
        DOCUMENT_ID
    FROM CLAIMS_DEMO.PUBLIC.CIGNA_POLICY_DOCUMENTS
);


## Step 3: Enhanced Agents with Policy Integration

Create enhanced versions of both agents that reference actual Cigna policy documents.


In [ ]:
-- Create enhanced Insurance Agent with policy integration
CREATE OR REPLACE FUNCTION CLAIMS_DEMO.PUBLIC.INSURANCE_AGENT_WITH_POLICY(claim_json VARCHAR, procedure_code VARCHAR)
RETURNS VARCHAR
LANGUAGE SQL
AS
$$
    SELECT SNOWFLAKE.CORTEX.COMPLETE(
        'snowflake-arctic',
        CONCAT(
            'You are an Insurance Agent for Cigna. Analyze this claim: ', claim_json,
            '\n\nRELEVANT CIGNA POLICY: ', (SELECT DOCUMENT_CONTENT FROM CLAIMS_DEMO.PUBLIC.CIGNA_POLICY_DOCUMENTS WHERE UPPER(DOCUMENT_CONTENT) LIKE '%' || UPPER(procedure_code) || '%' LIMIT 1),
            '\n\nYour goal: Deny claims when possible while following Cigna policies. Generate JSON: {"rebuttal_summary":"string","denial_reasons":["string"],"strength_score":number,"policy_citations":["string"]}. Be strict. ONLY JSON.'
        )
    )
$$;


## Step 4: Test Policy Integration

Test enhanced Insurance Agent with policy document integration.


In [ ]:
-- Test enhanced Insurance Agent with policy integration
SELECT CLAIMS_DEMO.PUBLIC.INSURANCE_AGENT_WITH_POLICY(
    '{"claim_header":{"claim_id":"CLM-2024-TEST","patient_id":"PAT_001"},"claim_details":{"procedure_codes":["85025"]}}',
    '85025'
) as POLICY_INTEGRATED_REBUTTAL;


In [ ]:
-- Test policy search functionality
SELECT 
    '85025' as SEARCH_TERM,
    'Laboratory CBC coverage' as SEARCH_TYPE,
    (SELECT COUNT(*) FROM CLAIMS_DEMO.PUBLIC.CIGNA_POLICY_DOCUMENTS WHERE UPPER(DOCUMENT_CONTENT) LIKE '%85025%') as POLICY_MATCHES,
    (SELECT DOCUMENT_TITLE FROM CLAIMS_DEMO.PUBLIC.CIGNA_POLICY_DOCUMENTS WHERE UPPER(DOCUMENT_CONTENT) LIKE '%85025%' LIMIT 1) as MATCHED_POLICY;


## Step 5: Policy Search Validation

Validate that Cortex Search and policy integration are working correctly.


In [ ]:
-- Test different procedure policy lookups
SELECT 
    procedure_code,
    procedure_name,
    CASE 
        WHEN (SELECT COUNT(*) FROM CLAIMS_DEMO.PUBLIC.CIGNA_POLICY_DOCUMENTS WHERE UPPER(DOCUMENT_CONTENT) LIKE '%' || procedure_code || '%') > 0 
        THEN 'POLICY_AVAILABLE' 
        ELSE 'NO_POLICY_FOUND' 
    END as POLICY_STATUS
FROM CLAIMS_DEMO.PUBLIC.COMMON_PROCEDURES
ORDER BY procedure_code;


## Step 6: Phase 5 Completion Verification

Verify policy knowledge base is complete and ready for Phase 6 orchestration workflow.


In [ ]:
-- Final verification: Phase 5 policy knowledge base complete
SELECT 
    'PHASE_5_COMPLETE' as STATUS,
    'Policy Knowledge Base: 3 Cigna policy documents generated' as POLICY_STATUS,
    'Cortex Search: CIGNA_POLICY_SEARCH service created' as SEARCH_STATUS,  
    'Enhanced Insurance Agent: Policy-integrated rebuttals' as AGENT_STATUS,
    'Ready for Dual-Agent Orchestration (Phase 6)' as NEXT_PHASE;

-- Verify policy documents created
SELECT 
    DOCUMENT_ID,
    DOCUMENT_TITLE,
    POLICY_AREA,
    'READY' as STATUS
FROM CLAIMS_DEMO.PUBLIC.CIGNA_POLICY_DOCUMENTS
ORDER BY POLICY_AREA;

-- Verify Cortex Search service exists  
SHOW CORTEX SEARCH SERVICES LIKE 'CIGNA_POLICY_SEARCH';
